In [ ]:
"import logging\nfrom pyspark.sql import functions as F\nfrom pyspark.sql.window import Window\nfrom pyspark.sql.types import IntegerType, StringType\n\n# Set up logging\nlogging.basicConfig(level=logging.INFO)\nlogger = logging.getLogger(__name__)\n\ntry:\n    # Load data from Unity Catalog tables\n    transactions_df = spark.table(\"genai_demo.sas.transactions\")\n    customers_df = spark.table(\"genai_demo.sas.customers\")\n\n    # Step 1: Valid Transactions Filtering\n    valid_txns_df = transactions_df.filter((F.col(\"Sales\") > 0) & (F.col(\"Product\") != \"\"))\n\n    # Step 2: Effective Price Calculation\n    trans_step2_df = valid_txns_df.withColumn(\"EffectivePrice\", F.col(\"Sales\") * (1 - F.col(\"Discount\") / 100))\n\n    # Step 3: Total Value Calculation\n    trans_step3_df = trans_step2_df.withColumn(\"TotalValue\", F.col(\"EffectivePrice\") * F.col(\"Quantity\"))\n\n    # Step 4: Full Data Join\n    full_data_df = trans_step3_df.join(customers_df, trans_step3_df.CustomerID == customers_df.CustomerID, \"left\") \\\n                                 .select(trans_step3_df[\"*\"], customers_df[\"Region\"], customers_df[\"JoinDate\"])\n\n    # Step 5: Tenure Days Calculation\n    trans_step5_df = full_data_df.withColumn(\"TenureDays\", F.datediff(F.col(\"TransDate\"), F.col(\"JoinDate\")))\n\n    # Step 6: Tenure Category Assignment\n    trans_step6_df = trans_step5_df.withColumn(\"TenureCategory\", \n                                               F.when(F.col(\"TenureDays\") < 180, \"New\")\n                                                .when(F.col(\"TenureDays\") < 365, \"Medium\")\n                                                .otherwise(\"Loyal\"))\n\n    # Step 7: High Value Flag\n    trans_step7_df = trans_step6_df.withColumn(\"HighValueFlag\", F.col(\"TotalValue\") > 2000)\n\n    # Step 8: Product Group Assignment\n    trans_step8_df = trans_step7_df.withColumn(\"ProductGroup\", \n                                               F.when(F.col(\"Product\").isin([\"A\", \"C\"]), \"Core\")\n                                                .otherwise(\"Non-Core\"))\n\n    # Step 9: Final Data Preparation\n    final_data_df = trans_step8_df\n\n    # Step 10: Z-score Standardization\n    windowSpec = Window.partitionBy(\"ProductGroup\")\n    standardized_df = final_data_df.withColumn(\"TotalValueZScore\", \n                                               (F.col(\"TotalValue\") - F.avg(\"TotalValue\").over(windowSpec)) / F.stddev(\"TotalValue\").over(windowSpec))\n\n    # Step 11: Outlier Detection\n    enhanced_final_data_df = standardized_df.withColumn(\"OutlierFlag\", F.abs(F.col(\"TotalValueZScore\")) > 2)\n\n    # Write the enhanced final data to Unity Catalog\n    spark.sql(\"DROP TABLE IF EXISTS genai_demo.sas.enhanced_final_data\")\n    enhanced_final_data_df.write.format(\"delta\").mode(\"overwrite\").saveAsTable(\"genai_demo.sas.enhanced_final_data\")\n\n    # Generate summary statistics by Region and Product Group\n    summary_stats_df = enhanced_final_data_df.groupBy(\"Region\", \"ProductGroup\") \\\n                                             .agg(F.mean(\"TotalValue\").alias(\"MeanTotalValue\"),\n                                                  F.sum(\"TotalValue\").alias(\"SumTotalValue\"),\n                                                  F.mean(\"Quantity\").alias(\"MeanQuantity\"),\n                                                  F.sum(\"Quantity\").alias(\"SumQuantity\"),\n                                                  F.mean(\"Sales\").alias(\"MeanSales\"),\n                                                  F.sum(\"Sales\").alias(\"SumSales\"))\n\n    # Write summary statistics to Unity Catalog\n    spark.sql(\"DROP TABLE IF EXISTS genai_demo.sas.summary_stats\")\n    summary_stats_df.write.format(\"delta\").mode(\"overwrite\").saveAsTable(\"genai_demo.sas.summary_stats\")\n\n    # Generate tenure category frequency by Region\n    tenure_freq_df = enhanced_final_data_df.groupBy(\"TenureCategory\", \"Region\").count()\n\n    # Write tenure frequency to Unity Catalog\n    spark.sql(\"DROP TABLE IF EXISTS genai_demo.sas.tenure_freq\")\n    tenure_freq_df.write.format(\"delta\").mode(\"overwrite\").saveAsTable(\"genai_demo.sas.tenure_freq\")\n\n    # Perform correlation analysis\n    correlation_df = enhanced_final_data_df.select(\"Sales\", \"Discount\", \"Quantity\", \"TotalValue\").corr()\n\n    # Write correlation analysis to Unity Catalog\n    spark.sql(\"DROP TABLE IF EXISTS genai_demo.sas.correlation_analysis\")\n    correlation_df.write.format(\"delta\").mode(\"overwrite\").saveAsTable(\"genai_demo.sas.correlation_analysis\")\n\n    # Generate summary by Outlier Flag\n    outlier_summary_df = enhanced_final_data_df.groupBy(\"OutlierFlag\") \\\n                                               .agg(F.mean(\"Sales\").alias(\"MeanSales\"),\n                                                    F.stddev(\"Sales\").alias(\"StdDevSales\"),\n                                                    F.mean(\"TotalValue\").alias(\"MeanTotalValue\"),\n                                                    F.stddev(\"TotalValue\").alias(\"StdDevTotalValue\"),\n                                                    F.mean(\"Quantity\").alias(\"MeanQuantity\"),\n                                                    F.stddev(\"Quantity\").alias(\"StdDevQuantity\"))\n\n    # Write outlier summary to Unity Catalog\n    spark.sql(\"DROP TABLE IF EXISTS genai_demo.sas.outlier_summary\")\n    outlier_summary_df.write.format(\"delta\").mode(\"overwrite\").saveAsTable(\"genai_demo.sas.outlier_summary\")\n\n    logger.info(\"ETL workflow completed successfully.\")\n\nexcept Exception as e:\n    logger.error(f\"An error occurred during the ETL process: {e}\")\n    raise\n"
